In [29]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import scipy
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [31]:
# Hyper parameters
num_classes = 10 # とりあえず10人で
num_epochs = 30
# batch_size = 128
batch_size = 64
learning_rate = 0.01

sequence_length = 1250 # 1セグメントあたりのサンプル数 250Hz * 5s
input_size = 1 # 特徴量の種類？
hidden_size = 100 # 論文でいうhidden layerはこれ?
num_layers = 2 # Two-layered lstm?

In [32]:
!pwd
!ls

/mnt/c/Users/grpro/workspace/grad_thesis/lstm_cnn_ensemble
CNN.ipynb   LSTM_colab.ipynb	 README.md  env
LSTM.ipynb  LSTM_tutorial.ipynb  data	    output.png


In [33]:
default_sample_rate = 2000
down_sample_rate = 250
ratio = default_sample_rate // down_sample_rate

In [34]:
# 10人分のdataframeを作る
radar_frame_list = []

for i in range(1, 11):
    wave_dem_2d = []
    file_path = "./data/radar_%02d.csv" % i
    radar_frame = pd.read_csv(file_path)
    wave = radar_frame.to_numpy().flatten()
    wave_dem = signal.decimate(wave, ratio)
    for i in range(len(wave_dem)):
      wave_dem_2d.append([wave_dem[i]])
    # wave_dem_ts = torch.from_numpy(wave_dem.astype(np.float32)).clone() # tensorにする
    print(i, len(wave), len(wave_dem_2d) // 900)

    remaining = len(wave_dem_2d)
    n = 0
    n_stop = 1250
    wave_segments = []

    while n_stop < len(wave_dem_2d):
        n_start = 0 + (1249 - (350 - 1)) * n
        n_stop = n_start + 1250
        tmp = []
        wave_segments.append(wave_dem_2d[n_start:n_stop])
        n += 1
    
    radar_frame_list.append(wave_segments)

151899 1215199 168
155587 1244699 172
150337 1202699 167
150768 1206149 167
152524 1220199 169
152731 1221849 169
158731 1269849 176
154643 1237149 171
162349 1298799 180
159781 1278249 177


In [35]:
wave_dem_2d[0][0]

1.938177274754611e-05

In [36]:
type(wave_dem_2d) # tensorにはしていない

list

In [37]:
wave_dem[0]

1.938177274754611e-05

In [38]:
print(radar_frame.to_numpy())

[[1.96265639e-05]
 [1.98858680e-05]
 [2.01462431e-05]
 ...
 [5.90767715e-06]
 [5.73961677e-06]
 [5.57643752e-06]]


In [39]:
labels = []
for i in range(len(radar_frame_list)):
    for j in range(len(radar_frame_list[i])):
        labels.append(i + 1)

labels_df = pd.Series(labels)
torch.tensor(labels_df)

tensor([ 1,  1,  1,  ..., 10, 10, 10])

In [40]:
df = pd.DataFrame(radar_frame_list)
tmp = df.to_numpy().flatten()
df = pd.Series(tmp).dropna()
tmp = df.to_numpy().flatten()

tmp_labels = labels_df.to_numpy().flatten()
for i in reversed(range(len(tmp))):
  if len(tmp[i]) != 1250:
    print(i, len(tmp[i]))
    tmp = np.delete(tmp, i)
    tmp_labels = np.delete(tmp_labels, i)

df = pd.Series(tmp)
labels_df = pd.Series(tmp_labels)

1722 482
1364 744
1192 1232
1016 632
846 425
676 469
508 938
341 788
168 700


In [41]:
df, type(df), type(df[0]), type(df[0][0]), type(df[0][0][0])

(0       [[-0.00010217575366496895], [-9.22608829183728...
 1       [[0.00011229699387508095], [0.0001032509100699...
 2       [[0.00018580354782126828], [0.0001857031428586...
 3       [[2.469195076897992e-05], [1.5866761530307943e...
 4       [[-7.67209241147911e-05], [-7.923320450820692e...
                               ...                        
 1709    [[-1.5671828712846213e-05], [-1.73393999462063...
 1710    [[1.5179027283521696e-05], [1.5135722189029168...
 1711    [[2.8651546425191015e-05], [2.854097153938563e...
 1712    [[-0.00016230754416965637], [-0.00016345281810...
 1713    [[3.255477277617704e-05], [3.270253213571334e-...
 Length: 1714, dtype: object,
 pandas.core.series.Series,
 list,
 list,
 numpy.float64)

In [42]:
labels_df

0        1
1        1
2        1
3        1
4        1
        ..
1709    10
1710    10
1711    10
1712    10
1713    10
Length: 1714, dtype: int64

In [43]:
class MyDataset(Dataset):
    def __init__(self, dataset, labels, root_dir, transform=None) -> None:
        # super().__init__()
        self.radar_heartbeat = dataset
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx = idx.tolist()
        # heartbeat_segment = torch.FloatTensor(self.radar_heartbeat[idx])
        # subject_label = torch.LongTensor(self.labels[idx])
        # if self.transform: # transform適用されず?
        # heartbeat_segment = self.transform(self.radar_heartbeat[idx]) # tensorに
        onehot_label = torch.eye(num_classes)[self.labels[idx] - 1] # one hot encording
        # onehot_label = self.transform(self.labels[idx]) # tensorに

        # return heartbeat_segment, subject_label
        # return torch.from_numpy(self.radar_heartbeat.to_numpy().copy()), torch.tensor(labels_df)    
        # return torch.stack(self.radar_heartbeat.to_numpy().tolist()), torch.tensor(labels_df)    
        # return self.radar_heartbeat[idx], labels_df[idx]
        return torch.tensor(self.radar_heartbeat[idx]), onehot_label



    def __len__(self):
        return len(self.radar_heartbeat)

In [44]:
dataset = MyDataset(df, labels_df, "./data/", transform=transforms.ToTensor())

In [45]:
seg, lab = next(iter(dataset))

In [46]:
type(seg), len(seg), lab

(torch.Tensor, 1250, tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [47]:
train_size = int(0.8 * len(df.values))
test_size = len(df.values) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
print(f"full: {len(dataset)} -> train: {len(train_set)}, test: {len(test_set)}")

full: 1714 -> train: 1371, test: 343


In [48]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [49]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    # print(x.size(), h0.size(), c0.size())
    # out, _ = self.rnn(x, h0)
    out, _ = self.lstm(x, (h0, c0))

    out = out[:, -1, :]

    out = self.fc(out)

    return out

In [50]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (signals, labels) in enumerate(train_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    labels = labels.to(device)

    # print(signals.size())
    outputs = model(signals)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch [{epoch+1}/`{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

/tmp/ipykernel_1334/2106716574.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(signals)


Epoch [1/`30], Step [22/22], Loss: 2.3133
Epoch [2/`30], Step [22/22], Loss: 2.2950
Epoch [3/`30], Step [22/22], Loss: 2.3154
Epoch [4/`30], Step [22/22], Loss: 2.3129
Epoch [5/`30], Step [22/22], Loss: 2.2959
Epoch [6/`30], Step [22/22], Loss: 2.2988
Epoch [7/`30], Step [22/22], Loss: 2.3042
Epoch [8/`30], Step [22/22], Loss: 2.2992
Epoch [9/`30], Step [22/22], Loss: 2.2873
Epoch [10/`30], Step [22/22], Loss: 2.3072
Epoch [11/`30], Step [22/22], Loss: 2.3284
Epoch [12/`30], Step [22/22], Loss: 2.3103
Epoch [13/`30], Step [22/22], Loss: 2.2951
Epoch [14/`30], Step [22/22], Loss: 2.3160
Epoch [15/`30], Step [22/22], Loss: 2.3107
Epoch [16/`30], Step [22/22], Loss: 2.3052
Epoch [17/`30], Step [22/22], Loss: 2.2970
Epoch [18/`30], Step [22/22], Loss: 2.2893
Epoch [19/`30], Step [22/22], Loss: 2.3189
Epoch [20/`30], Step [22/22], Loss: 2.2979
Epoch [21/`30], Step [22/22], Loss: 2.3069
Epoch [22/`30], Step [22/22], Loss: 2.3047
Epoch [23/`30], Step [22/22], Loss: 2.2822
Epoch [24/`30], Step

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1) # softmax通してないけどクラス分の出力から最大値選べばOK
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [52]:
outputs.shape, labels

(torch.Size([27, 10]),
 tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        